# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [5]:
%load_ext autoreload
%autoreload 2
from helpers.irp_integration import IRPClient
import json

irp_client = IRPClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Configurations

In [8]:
server_name = "databridge-1"
peril = "USEQ"
edm_name = f"{peril}_Automated" # ensure unique within IRP
treaty_name = f"{peril}_Treaty"
edm_data_version = "22"
portfolio_name = f"{peril}_Automated" # ensure unique within the edm
geohaz_version = "22.0"
accounts_file_name = f"Modeling_202503_Moodys_{peril}_Account.csv"
locations_file_name = f"Modeling_202503_Moodys_{peril}_Location.csv"
mapping_file_name = "mapping.json"
analysis_job_name = f"{peril} Analysis"
analysis_profile_name = f"DLM {peril} v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
event_rate_scheme_name = "RMS 2025 Stochastic Event Rates"
analysis_tag_name = f"{peril}_TAG"
rdm_name = f"{peril}_Automated_RDM"

### Create EDM(s)

In [9]:
job_ids = irp_client.edm.submit_create_edm_jobs(
    [
        {
            "edm_name": edm_name,
            "server_name": server_name
        }
    ]
)
print(json.dumps(irp_client.job.poll_risk_data_job_batch_to_completion(job_ids), indent=2))

Polling batch risk data job ids: 23130176
Polling batch risk data job ids: 23130176
Polling batch risk data job ids: 23130176
Polling batch risk data job ids: 23130176
Polling batch risk data job ids: 23130176
Polling batch risk data job ids: 23130176
[
  {
    "jobId": "23130176",
    "userName": "Partner - PremiumIQ key",
    "status": "FINISHED",
    "submittedAt": "2025-11-06T04:38:44.970Z",
    "startedAt": "2025-11-06T04:38:47Z",
    "endedAt": "2025-11-06T04:40:22Z",
    "name": "USEQ_Automated",
    "type": "CREATE_EDM",
    "progress": 100,
    "priority": "medium",
    "entitlement": "RI-RISKMODELER",
    "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
    "details": {
      "resources": [],
      "summary": "CREATE_EDM is successful"
    },
    "warnings": []
  }
]


### Create Portfolio(s)

In [10]:
portfolio_ids = irp_client.portfolio.create_portfolios(
    [
        {
            "edm_name": edm_name,
            "portfolio_name": portfolio_name,
            "portfolio_number": portfolio_name,
            "description": f"{portfolio_name} created via IRP Notebook Framework"
        }
    ]
)
print(f"Created {len(portfolio_ids)} portfolios with IDs: {portfolio_ids}")

Created 1 portfolios with IDs: [1]


### MRI Import

In [12]:
import_response = irp_client.mri_import.import_from_files(
    edm_name=edm_name,
    portfolio_name=portfolio_name,
    accounts_file=accounts_file_name,
    locations_file=locations_file_name,
    mapping_file="mapping.json"
)

print(json.dumps(import_response, indent=2))

Creating AWS bucket...
AWS bucket created!
Access credentials received
Uploading file to c:\Users\hello\Desktop\irp-notebook-framework\workspace\workflows\_Tools\files\working_files\Modeling_202503_Moodys_USEQ_Account.csv s3...
File uploaded!
Uploading file to c:\Users\hello\Desktop\irp-notebook-framework\workspace\workflows\_Tools\files\working_files\Modeling_202503_Moodys_USEQ_Location.csv s3...
File uploaded!
Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23130189
Workflow status: PENDING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23130189
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23130189
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23130189
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-

### Create Treaty

In [13]:
treaty_ids = irp_client.treaty.create_treaties(
    [
        {
            "edm_name": edm_name,
            "treaty_name": treaty_name,
            "treaty_number": treaty_name,
            "treaty_type": "Working Excess",
            "per_risk_limit": 30000000.00,
            "occurrence_limit": 90000000.00,
            "attachment_point": 2000000.00,
            "inception_date": "2025-01-01T00:00:00.000Z",
            "expiration_date": "2030-01-01T00:00:00.000Z",
            "currency_name": "US Dollar",
            "attachment_basis": "Losses Occurring",
            "attachment_level": "Location",
            "pct_covered": 100.0,
            "pct_placed": 95.00,
            "pct_share": 100.00,
            "pct_retention": 100.00,
            "premium": 0.00,
            "num_reinstatements": 99,
            "pct_reinstatement_charge": 0.00,
            "aggregate_limit": 0.00,
            "aggregate_deductible": 0.00,
            "priority": 1
        }
    ]
)
print(f"Created {len(treaty_ids)} with IDs: {treaty_ids}")

Created 1 with IDs: [1]


### Upgrade EDM Version

In [14]:
upgrade_job_ids = irp_client.edm.submit_upgrade_edm_data_version_jobs(
    [
        {
            "edm_name": edm_name,
            "edm_version": edm_data_version
        }
    ]
)
print(json.dumps(irp_client.edm.poll_data_version_upgrade_job_batch_to_completion(upgrade_job_ids), indent=2))

Polling batch upgrade edm version job ids: 23130199
Polling batch upgrade edm version job ids: 23130199
Polling batch upgrade edm version job ids: 23130199
[
  {
    "id": 23130199,
    "userName": "Partner - PremiumIQ key",
    "status": "FINISHED",
    "submitTime": "2025-11-06T04:57:27.167Z",
    "startTime": "2025-11-06T04:57:28Z",
    "endTime": "2025-11-06T04:57:46Z",
    "name": "USEQ_Automated",
    "type": "EDM_DATA_UPGRADE",
    "progress": 100,
    "priority": "medium",
    "entitlement": "RI-RISKMODELER",
    "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
    "jobs": [
      {
        "id": "93822913-bcac-42a1-87ca-8c2062a1f485",
        "taskId": 1,
        "workflowId": 23130199,
        "status": "Succeeded",
        "submitTime": "2025-11-06T04:57:27.919Z",
        "createdAt": "2025-11-06T04:57:27.151Z",
        "name": "EDM_DATA_UPGRADE",
        "input": {
          "name": "EDM_DATA_UPGRADE"
        },
        "output": {
          "databaseName": "USEQ_

### GeoHaz

In [15]:
geohaz_job_ids = irp_client.portfolio.submit_geohaz_jobs(
    [
        {
            "edm_name": edm_name,
            "portfolio_name": portfolio_name,
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        }
    ]
)
print(json.dumps(irp_client.portfolio.poll_geohaz_job_batch_to_completion(geohaz_job_ids), indent=2))

Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job ids: 23130200
Polling batch geohaz job 

### Execute Multiple Analyses

In [ ]:
analysis_job_ids = irp_client.analysis.submit_portfolio_analysis_jobs(
    [
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2023 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2023 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        },
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2025 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2025 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        }
    ]
)
analysis_batch_response = irp_client.analysis.poll_analysis_job_batch_to_completion(analysis_job_ids)
print(json.dumps(analysis_batch_response, indent=2))

### Group Analyses

In [ ]:
analysis_names = ["CBHU 2023 Rates", "CBHU 2025 Rates"]
grouping_job_ids = irp_client.analysis.submit_analysis_grouping_jobs(
    [
        {
            "group_name": "CBHU_Group",
            "analysis_names": analysis_names
        }
    ]
)
grouping_response = irp_client.analysis.poll_analysis_grouping_job_batch_to_completion(grouping_job_ids)
print(json.dumps(grouping_response, indent=2))

### Export Analyses to RDM

In [ ]:
analysis_names = ["CBHU Group", "CBHU 2023 Rates", "CBHU 2025 Rates"]
rdm_export_response = irp_client.rdm.export_analyses_to_rdm(
    server_name=server_name,
    rdm_name=rdm_name,
    analysis_names=analysis_names
)
print(json.dumps(rdm_export_response, indent=2))

### Delete EDM

In [ ]:
print(json.dumps(irp_client.edm.delete_edm(edm_name), indent=2))